In [1]:
import sys
is_training = True
Py3 = sys.version_info[0] == 3
CUDNN = "cudnn"
num_steps = 35
def _get_lstm_cell():
    return tf.contrib.rnn.LSTMBlockCell(hidden_size, forget_bias=0.0)

def make_cell():
    cell = _get_lstm_cell()
    return cell

hidden_size = 1500
batch_size = 20

def _read_words(filename):
  with tf.gfile.GFile(filename, "r") as f:
    if Py3:
      return f.read().replace("\n", "<eos>").split()
    else:
      return f.read().decode("utf-8").replace("\n", "<eos>").split()

def _build_vocab(filename):
  data = _read_words(filename)

  counter = collections.Counter(data)
  count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))

  words, _ = list(zip(*count_pairs))
  word_to_id = dict(zip(words, range(len(words))))

  return word_to_id

def _file_to_word_ids(filename, word_to_id):
  data = _read_words(filename)
  return [word_to_id[word] for word in data if word in word_to_id]

def ptb_raw_data(data_path=None):
  """Load PTB raw data from data directory "data_path".
  Reads PTB text files, converts strings to integer ids,
  and performs mini-batching of the inputs.
  The PTB dataset comes from Tomas Mikolov's webpage:
  http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz
  Args:
    data_path: string path to the directory where simple-examples.tgz has
      been extracted.
  Returns:
    tuple (train_data, valid_data, test_data, vocabulary)
    where each of the data objects can be passed to PTBIterator.
  """

  train_path = os.path.join(data_path, "train.txt")
  valid_path = os.path.join(data_path, "valid.txt")
  test_path = os.path.join(data_path, "test.txt")

  word_to_id = _build_vocab(train_path)
  train_data = _file_to_word_ids(train_path, word_to_id)
  valid_data = _file_to_word_ids(valid_path, word_to_id)
  test_data = _file_to_word_ids(test_path, word_to_id)
  vocabulary = len(word_to_id)
  return train_data, valid_data, test_data, vocabulary


def data_type():
  return tf.float32 if 1 else tf.float32

import tensorflow as tf

import os
import collections

raw_data = ptb_raw_data("simple-examples/")
train_data, valid_data, test_data, _ = raw_data

_input = tf.placeholder(tf.int32, shape=(batch_size, num_steps), name="input")
_targets = tf.placeholder(tf.int32, shape=(batch_size, num_steps), name="targets")
vocab_size = 10000
keep_prob = 0.35
num_layers = 2

init_scale = 0.04
initializer = tf.random_uniform_initializer(-init_scale,init_scale)
max_epoch = 14
lr_decay = 1/1.15
lrlr = 1.

with tf.variable_scope('MODEL', initializer=initializer,reuse=False):
    learning_rate = tf.Variable(1., trainable=False)
    embedding = tf.get_variable("embedding", [vocab_size, hidden_size], dtype=data_type())
    inputs = tf.nn.embedding_lookup(embedding, _input)
    val_inputs = inputs
    inputs = tf.nn.dropout(inputs, keep_prob)
    val_cell = [make_cell() for _ in range(num_layers)]
    cell = [tf.contrib.rnn.DropoutWrapper(c,output_keep_prob=keep_prob) for c in val_cell]
    the_cell = tf.contrib.rnn.MultiRNNCell(cell, state_is_tuple=True)
    #val_cell = tf.contrib.rnn.MultiRNNCell(val_cell)
    initial_state = the_cell.zero_state(batch_size, data_type())
    #inputs = tf.unstack(inputs, num=num_steps, axis=1)
    #outputs, final_state = tf.nn.static_rnn(the_cell, inputs,initial_state=initial_state)
    #cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)

    outputs, final_state = tf.nn.dynamic_rnn(the_cell, inputs, initial_state=initial_state,dtype=data_type())
    #val_outputs, val_final_state = tf.nn.dynamic_rnn(val_cell, val_inputs, initial_state=initial_state,dtype=tf.float32)
    #val_output = tf.reshape(tf.concat(val_outputs, 1), [-1, hidden_size])
    output = tf.reshape(tf.concat(outputs, 1), [-1, hidden_size])
    softmax_w = tf.get_variable("softmax_W", [hidden_size, vocab_size], dtype=data_type())
    softmax_b = tf.get_variable("softmax_B", [vocab_size], dtype=data_type())
    logits = tf.nn.xw_plus_b(output, softmax_w, softmax_b)
    logits = tf.reshape(logits, [batch_size, num_steps, vocab_size])
    #val_logits = tf.nn.xw_plus_b(val_output, softmax_w, softmax_b)
    #val_logits = tf.reshape(val_logits, [batch_size, num_steps, vocab_size])

    loss = tf.contrib.seq2seq.sequence_loss(
            logits,
            _targets,
            tf.ones([batch_size, num_steps], dtype=data_type()),
            average_across_timesteps=False,
            average_across_batch=True)
    cost = tf.reduce_sum(loss)
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars),10)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.apply_gradients(zip(grads, tvars))

    new_lr = tf.placeholder(tf.float32, shape=[])
    lr_update = tf.assign(learning_rate, new_lr)
    
with tf.variable_scope('MODEL', initializer=initializer,reuse=True):
    val_cell = tf.contrib.rnn.MultiRNNCell(val_cell)
    val_outputs, val_final_state = tf.nn.dynamic_rnn(val_cell, val_inputs, initial_state=initial_state,dtype=tf.float32)
    val_output = tf.reshape(tf.concat(val_outputs, 1), [-1, hidden_size])
    val_logits = tf.nn.xw_plus_b(val_output, softmax_w, softmax_b)
    val_logits = tf.reshape(val_logits, [batch_size, num_steps, vocab_size])

    val_loss = tf.contrib.seq2seq.sequence_loss(
            val_logits,
            _targets,
            tf.ones([batch_size, num_steps], dtype=data_type()),
            average_across_timesteps=False,
            average_across_batch=True)
    val_cost = tf.reduce_sum(val_loss)

config = tf.ConfigProto(
    gpu_options = tf.GPUOptions(allow_growth=True,visible_device_list="0"),
    device_count = {'GPU': 1},

#    log_device_placement = True                                                                                     \
                                                                                                                      
)
sess = tf.Session(config = config)
sess.run(tf.global_variables_initializer())

import numpy as np

data_len = len(train_data)
batch_len = data_len // num_steps
num_batches = int(data_len / (batch_size * num_steps))
xdata = train_data[:num_batches * batch_size * num_steps]
ydata = np.copy(xdata)
ydata[:-1] = xdata[1:]
ydata[-1] = xdata[0]
x_batches = np.split(np.asarray(xdata).reshape(batch_size, -1), num_batches, 1)
y_batches = np.split(np.asarray(ydata).reshape(batch_size, -1), num_batches, 1)
valid_len = len(valid_data)
num_batches = int(valid_len / (batch_size * num_steps))
vdata = valid_data[:num_batches * batch_size * num_steps]
vydata = np.copy(vdata)
vydata[:-1] = vdata[1:]
vydata[-1] = vdata[0]
x_val_batches = np.split(np.asarray(vdata).reshape(batch_size, -1), num_batches, 1)
y_val_batches = np.split(np.asarray(vydata).reshape(batch_size, -1), num_batches, 1)


test_len = len(test_data)
num_batches = int(test_len / (batch_size * num_steps))
tdata = test_data[:num_batches * batch_size * num_steps]
tydata = np.copy(tdata)
tydata[:-1] = tdata[1:]
tydata[-1] = tdata[0]
x_test_batches = np.split(np.asarray(tdata).reshape(batch_size, -1), num_batches, 1)
y_test_batches = np.split(np.asarray(tydata).reshape(batch_size, -1), num_batches, 1)

/home/ec2-user/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
Vars = np.load("Large_LSTM.npy")
for k,v in enumerate(tvars):
    sess.run(tf.assign(v,Vars[k]))
W1 = sess.run(tvars[0])
W2 = sess.run(tvars[-2])
W2 = W2.transpose()

IOError: [Errno 2] No such file or directory: 'Large_LSTM.npy'

In [3]:
# VALIDATE #
stateb = sess.run(initial_state)
t = np.arange(len(x_test_batches))
cc = 0.0
cnt = 0
for k,idx in enumerate(t):
    feed_dict = {}
    for i, (c, h) in enumerate(initial_state):
        feed_dict[c] = stateb[i].c
        feed_dict[h] = stateb[i].h
    feed_dict[_input] = x_test_batches[idx]
    feed_dict[_targets] = y_test_batches[idx]    
    _c,stateb = sess.run([val_cost,val_final_state],feed_dict=feed_dict)
    cc += _c
    cnt += num_steps

print np.exp(cc/cnt)


78.32328562594896


In [4]:
def Compress(X,compression_rate, Weights, top_protected_ratio, n_clusters, k_recon_iter):
    """
    X : Matrix to compress with shape vocab_size x hidden_size, assumed the rows are already sorted by
        frequency of the word
    compression_rate : scalar, compression rate of the matrix X
    weights: weights of each vocaburary in X with shape 1xvocab_size
    top_protected_ratio: scalr, percentage of vocaburaries not approximated at all
    k_mean_iter: number of iterations of K-reconstruction process
    return values:
    
    """
    def weighted_svd(A, weight, k):
        # weight is for with each row of A 
        # solve min_{U, V} ||WUV^T - WA||^2,   W = diag(sqrt(weight))
        W = np.diag(weight**(0.5))
        QA = np.matmul(W,A)

        m_square = np.dot(np.transpose(QA),QA)
        u, s, v = np.linalg.svd(m_square, full_matrices=True)
        v = np.transpose(v)
        if(k > 0):
            U = np.dot(np.diag(1/(weight**0.5)),np.dot(QA,v[:, 0:k]))
            V = np.transpose(v[:, 0:k])
            return U,V
        else:
            return v


    ## Initial Setup
    hidden_size = X.shape[1]
    entries_to_take = int(X.shape[0] * top_protected_ratio)
    X_sub = X[entries_to_take:,:]
    Weight_sub = Weights[entries_to_take:]
    target_rate = compression_rate / ( 1- top_protected_ratio)
    total_rank = (np.prod(X_sub.shape) * target_rate)/(X_sub.shape[0] + hidden_size)
    budget_rate = n_clusters * target_rate
    # Create initial basis and assignment
    vocab_size = X_sub.shape[0]
    idx = np.arange(vocab_size)
    initial_labels = np.asarray([np.random.randint(n_clusters) for p in range(vocab_size)])
    chunk_size = int(float(X_sub.shape[0])/n_clusters)
    for k in range(n_clusters):
        initial_labels[k*chunk_size:(k+1)*chunk_size] = k
    total_freq = float(sum(Weight_sub))
    basis = []
    ranks = []
    budgets = []
    for c in range(n_clusters):
        vecs = X_sub[np.where(initial_labels == c)[0]]
        weight_for_this_cluster = Weight_sub[np.where(initial_labels == c)[0]]
        local_rate = budget_rate * sum(weight_for_this_cluster)/total_freq
        rank_for_this_cluster = int(np.prod(vecs.shape) * local_rate / (vecs.shape[0] + hidden_size))
        #rank_for_this_cluster = int(sum(weight_for_this_cluster)/total_freq * total_rank)
        ranks.append(rank_for_this_cluster)
        print rank_for_this_cluster
        #rank_for_this_cluster = int(np.where(np.abs(vecs) > th)[0].shape[0]/(vecs.shape[0] + hidden_size))
        #ranks.append(int(np.where(np.abs(vecs) > th)[0].shape[0]/(vecs.shape[0] + hidden_size)))
    new_assignment = initial_labels
    ranks_for_each_param= []
    for i in range(X_sub.shape[0]):
        ranks_for_each_param.append(ranks[new_assignment[i]])
    minimal_rank = min(ranks)
    maximal_rank = max(ranks)
    for c in range(n_clusters):
        vecs = X_sub[np.where(initial_labels == c)[0]]
        weight_for_this_cluster = Weight_sub[np.where(initial_labels == c)[0]]
        B = weighted_svd(vecs,np.log(weight_for_this_cluster)+1e-20,0)
        basis.append(B)

    # Iteratively assign to the cluster with best approximation
    sample_order = np.arange(vocab_size)
    for i in range(k_recon_iter):

        recon_err = []

        for cc,x in enumerate(X_sub):
            projs = []
            rr = ranks_for_each_param[cc]
            for k,c in enumerate(basis):
                rru = min(ranks[k],rr)
                projs.append(np.linalg.norm((np.matmul(c[:rru,:].transpose(),np.matmul(c[:rru,:],x.transpose()))).transpose() - x))

            recon_err.append(projs)

        errs = np.stack(recon_err)
        #return errs,recon_err
        err = sum(np.min(errs,1))
        #print err

        prev_assignment = np.copy(new_assignment)

        temp_assignment = np.argmin(errs,1)
        sample_order = np.random.permutation(sample_order)
        selected_index = sample_order[:int(vocab_size*0.1)]
        new_assignment[selected_index] = temp_assignment[selected_index]
        #print sum(new_assignment == prev_assignment)
        if(sum(new_assignment == prev_assignment) == vocab_size):
            break
        basis = []
        for c in range(n_clusters):
            vecs = X_sub[np.where(new_assignment == c)[0]]
            weight_for_this_cluster = Weight_sub[np.where(initial_labels == c)[0]]
            rank = ranks[c]
            B = weighted_svd(vecs,np.log(weight_for_this_cluster)+1e-20,0)
            basis.append(B)

    clusters = []
    sequences = []
    final_weights = []
    for i in range(n_clusters):
        clusters.append(X_sub[np.where(new_assignment == i)[0]])
        sequences.append(np.where(new_assignment == i)[0]+entries_to_take)
        final_weights.append(Weight_sub[np.where(new_assignment == i)[0]])


    approximated_matrice = []
    Low_rank_matrice = []


    for k,p in enumerate(clusters):
        rank = ranks[k]
        #rank = max(1,int(parameter_ratio[k] * total_budgets/(p.shape[0] + hidden_size)))
        # Don't approximate if the cluster is too small, which is unlikely to happen
        if(p.shape[0] < 10):
            rank = p.shape[0]

        weight_for_this_cluster = final_weights[k]

        U,V = weighted_svd(p,np.log(weight_for_this_cluster)+1e-20,maximal_rank)
#        new_W1 = np.zeros(U.shape)
#        new_W2 = np.zeros(V.shape)
#        for i in range(U.shape[0]):
#            for j in range(U.shape[1]):
#                new_W1[i][j] = my_fp6(binary(U[i][j]))
#                new_W1[i][j] = np.float32(np.float
#        for i in range(V.shape[0]):
#            for j in range(V.shape[1]):
#                new_W2[i][j] = my_fp6(binary(V[i][j]))
#        U = new_W1
#        V = new_W2
#        print(np.unique(U))
#        print(np.unique(V))
#        U = np.float32(np.float16(U))
#        V = np.float32(np.float16(V))
#        recovery = np.matmul(new_W1,new_W2)
        recovery = np.matmul(U,V)
        approximated_matrice.append(recovery)
        Low_rank_matrice.append([U,V])


    new_X = np.zeros(X.shape)

    tt = sum(ranks) * hidden_size
    for i in range(n_clusters):
        print np.where(new_assignment == i)[0].shape[0]
    #return sequences,Low_rank_matrice,new_X,entries_to_take
    for kk,s in enumerate(sequences):
        L_U,L_V = Low_rank_matrice[kk]
        print s.shape
        #print L_U.shape,L_V.shape
        for k,p in enumerate(s):
            #new_X[p] = approximated_matrice[kk][k]
            rrr = min(ranks_for_each_param[p-entries_to_take],ranks[kk])
            #print rrr
            tt += rrr
            new_X[p] = np.matmul(L_U[k][:rrr],L_V[:rrr,:])
    new_X[:entries_to_take,:] = X[:entries_to_take,:]
    print tt,np.prod(X.shape),tt/float(np.prod(X.shape))

    return new_X

In [5]:
# GET FREQ INFORMATION FROM TRAIN DATA
vocab_size = 10000
a = np.zeros(vocab_size)
for t in train_data:
    a[t] += 1
a = np.asarray(a)

In [35]:
import time
start = time.time()
Rec_W1 = Compress(W1,0.10,a,0.05,5,5)
# Rec_W2 = Compress(W2,0.10,a,0.05,5,5)
end = time.time()

270
82
43
27
18
2858
2112
1721
1544
1265
(2858,)
(2112,)
(1721,)
(1544,)
(1265,)
1495623 15000000 0.0997082


In [36]:
    sess.run(tf.assign(tvars[0],Rec_W1))
    sess.run(tf.assign(tvars[-2],W2.transpose()))
    # VALIDATE #
    stateb = sess.run(initial_state)
    t = np.arange(len(x_test_batches))
    cc = 0.0
    cnt = 0
    for k,idx in enumerate(t):
        feed_dict = {}
        for i, (c, h) in enumerate(initial_state):
            feed_dict[c] = stateb[i].c
            feed_dict[h] = stateb[i].h
        feed_dict[_input] = x_test_batches[idx]
        feed_dict[_targets] = y_test_batches[idx]    
        _c,stateb = sess.run([val_cost,val_final_state],feed_dict=feed_dict)
        cc += _c
        cnt += num_steps

    print np.exp(cc/cnt)

79.91794159056336


In [16]:
np.save(open('W1.npy','wb'),W1)
np.save(open('freqs','wb'),a)

In [33]:
Rec_W1_ = np.load(open('W1.zip.npy'))

In [34]:
    sess.run(tf.assign(tvars[0],Rec_W1_))
    sess.run(tf.assign(tvars[-2],W2.transpose()))
    # VALIDATE #
    stateb = sess.run(initial_state)
    t = np.arange(len(x_test_batches))
    cc = 0.0
    cnt = 0
    for k,idx in enumerate(t):
        feed_dict = {}
        for i, (c, h) in enumerate(initial_state):
            feed_dict[c] = stateb[i].c
            feed_dict[h] = stateb[i].h
        feed_dict[_input] = x_test_batches[idx]
        feed_dict[_targets] = y_test_batches[idx]    
        _c,stateb = sess.run([val_cost,val_final_state],feed_dict=feed_dict)
        cc += _c
        cnt += num_steps

    print np.exp(cc/cnt)

82.49799848030283


In [37]:
W1.shape

(10000, 1500)